In [ ]:
########################################################################
# Date: Sept 2023
# Author: Astrid ALsema
# Purpose: rerun Spotlight spatial feature plot with cell proportions without HE image as background.

# Required inputs:
# - indir: path to 1.datasets.rds: the Visium data of MS lesions, a list of samples directed imported after spaceranger
# - path_to_decnv_matrices: this contains tables with the estimated cell type proportions per sample  
# as "ST_ID, "_celltype_prop.csv"
# - outdir: path to store plots

# Output:
# - spatial feature plot with estimated cell type proportion
########################################################################

In [5]:
rm(list=ls())
library(tidyr)
library(ggplot2)
library(Matrix)
library(data.table)
library(Seurat)
library(dplyr)
library(RColorBrewer)

outdir <- "YOUR_OUTDIR"
indir <- "RData/seurat/"
path_to_decnv_matrices <- "RData/SPOTlight/celltype_prop/"

# load ST data in list
ST_ls <- readRDS(paste0(indir,"1.datasets.rds")) # loop over all samples

for (i in 1:length(ST_ls)){
    ST <- ST_ls[[i]]
    ST_ID <- unique(ST$orig.ident)
    print(paste("analysing", ST_ID))

    # filter low-content spots
    print("filtering spots with less than 100 counts or features. From:")
    print(ncol(ST))
    ST <-subset(ST, subset = nCount_Spatial>100)
    ST <-subset(ST, subset = nFeature_Spatial>100)
    print("to")
    print(ncol(ST))

    decon_df <- read.csv(paste0(path_to_decnv_matrices, ST_ID, "_celltype_prop.csv"))

    # add deconvolution results to the meta-data of seurat.

    ST@meta.data <- ST@meta.data %>%
      tibble::rownames_to_column("barcodes") %>%
      dplyr::left_join(decon_df, by = "barcodes") %>%
      tibble::column_to_rownames("barcodes")

    ### make plots 
    pdf(paste0(outdir, ST_ID, "_spatialplot_proportions_v2.pdf"))
    print(Seurat::SpatialFeaturePlot(
      object = ST,
      image.alpha = 0,
      stroke = NA,
      features = c("Oligodendrocytes", "OPCs", "GM.astrocytes", "WM.astrocytes"),
      alpha = c(0.1, 1)))
    print(Seurat::SpatialFeaturePlot(
      object = ST,
      image.alpha = 0,
      stroke = NA,
      features = c("ExNeurons", "InhNeurons", "microglia.macrophages", "Vasculature"),
      alpha = c(0.1, 1)))
    dev.off()
    }